# Llama 3.2 90B fine tuning with large data set

In [1]:
import unsloth
import os
import numpy as np
import pandas as pd

import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer, AutoTokenizer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, FastVisionModel
from datasets import Dataset
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments
from transformers import BitsAndBytesConfig

# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
from huggingface_hub import login
 
login(
  token="<HF_API_KEY_REMOVED>", 
  add_to_git_credential=True
)

Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [3]:
max_seq_length = 12000

def load_model():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,  
        bnb_4bit_compute_dtype=torch.float16,  
        bnb_4bit_use_double_quant=True,  
        bnb_4bit_quant_type="nf4", 
    )

    model, tokenizer = FastVisionModel.from_pretrained(
        model_name="unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit",
        max_seq_length=max_seq_length,
        quantization_config=bnb_config, 
        device_map="auto" 
    )

    tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit")
    
    model = FastVisionModel.get_peft_model(
        model,

        finetune_vision_layers     = False,
        finetune_language_layers   = True,
        finetune_attention_modules = True,
        finetune_mlp_modules       = True,
        
        r=16,
        lora_alpha=16,
        lora_dropout=0,
        target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
        use_rslora=True,
        use_gradient_checkpointing="unsloth",
        random_state = 32,
        loftq_config = None,
    )
    return model, tokenizer

In [4]:
def create_trainer(model, tokenizer, training_data, max_steps):
    # def collate_fn(examples):
    #     processed_examples = [example['text'] for example in examples]
    
    #     batch = tokenizer(
    #         text=processed_examples, 
    #         images=None, 
    #         return_tensors="pt", 
    #         padding=True
    #     )
    
    #     labels = batch["input_ids"].clone()
    #     labels[labels == tokenizer.tokenizer.pad_token_id] = -100
    
    #     batch["labels"] = labels
    
    #     return batch
    
    # 2 alternative = use Out of the box data collator
    from transformers import DataCollatorForLanguageModeling
    collate_fn = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    
    training_arguments = UnslothTrainingArguments(
        learning_rate=3e-4,
        embedding_learning_rate = 3e-5,
        lr_scheduler_type="linear",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 128,
        num_train_epochs=40,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        # max_steps=max_steps,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=150,
        output_dir="output",
        seed=0,
        save_total_limit=3,
    )

    if max_steps is not None:
        training_arguments.max_steps = max_steps
    
    return UnslothTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=training_data,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=10,
        packing=True,
        args=training_arguments,
        data_collator = collate_fn,
    )

In [5]:
import numpy as np
from utils.similarity import calculate_metrics
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import torch

log_dir = 'output/runs'

VIEWPORT_SIZES = {
    'DESKTOP': {'width': 1440, 'height': 900},
    'TABLET': {'width': 834, 'height': 1210},
    'MOBILE': {'width': 393, 'height': 852},
}

def add_image_to_tensorboard(name, step, img_path):
    image = Image.open(img_path)
    image = image.convert('RGB')
    image_array = np.array(image)
    image_tensor = torch.from_numpy(image_array)
    image_tensor = image_tensor.permute(2, 0, 1)
    image_tensor = image_tensor.float() / 255.0
    
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_image(name, image_tensor, step)
    
def add_text_to_tensorboard(name, step, text):
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_text(name, text, step)

def postprocess_text(preds, labels):
    preds = [pred.strip().replace('<unk>', '') for pred in preds]
    labels = [[label.strip().replace('<unk>', '')] for label in labels]

    return preds, labels

def compute_metrics(decoded_predictions, decoded_labels, steps):
    similarity_scores = []
    perceptual_losses = []
    index = 1
    
    for prediction, label in zip(decoded_predictions, decoded_labels):
        prediction = prediction.replace(tokenizer.eos_token, '')
        
        add_text_to_tensorboard(f'valid_{index}_label_text', steps, label)
        add_text_to_tensorboard(f'valid_{index}_prediction_text', steps, prediction)
        
        metrics = calculate_metrics(prediction, label, 
                                    VIEWPORT_SIZES['MOBILE']['width'], VIEWPORT_SIZES['MOBILE']['height'])
        
        if metrics is not None:
            similarity_scores.append(metrics['similarity'])
            perceptual_losses.append(metrics['perceptual_loss'])
            
            add_image_to_tensorboard(f'valid_{index}_expectation', steps, metrics['expected_screenshot_path'])
            add_image_to_tensorboard(f'valid_{index}_prediction', steps, metrics['predicted_screenshot_path'])
        
        index += 1

    results = {
        "similarity": float(np.mean(similarity_scores)),
        "perceptual_loss": float(np.mean(perceptual_losses)),
    }
    
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_scalar('similarity', results['similarity'], steps)
    writer.add_scalar('perceptual_loss', results['perceptual_loss'], steps)
    
    print("Similarity:", results['similarity'])
    print("Perceptual loss:", results['perceptual_loss'])

    return results

def test_prediction(model, data, steps):
    answers = []
    labels = []
    print("Generating predictions...")
    for row in data:
        inputs = tokenizer(
        [
            data_prompt.format(
                #instructions
                row['svg'],
                #answer
                "",
            )
        ], return_tensors = "pt").to("cuda")
        
        outputs = model.generate(**inputs, max_new_tokens = max_seq_length, use_cache = True)
        answer = tokenizer.batch_decode(outputs)
        answers.append(answer[0].split("### Response:")[-1])
        labels.append(row['html'])

    print("Computing metrics...")
    return compute_metrics(answers, labels, steps)

In [6]:
!rm -rf output

In [ ]:
!apt install zip -y
!rm -rf data-rb-large
!mkdir -p data-rb-large
!wget "https://www.dropbox.com/scl/fi/868hi141d4hrzwntxxih3/data-rb-large-with-tokenizer.zip?rlkey=md8iq2htminafzqcol3tzfbva&dl=1" -O model.zip
!unzip model.zip -d data-rb-large

!rm -rf data-rb-validate
!mkdir -p data-rb-validate
!wget "https://www.dropbox.com/scl/fi/5szml8y5l248mcabj9rqg/verify-dataset.zip?rlkey=se33rwtxgngn0ts1i0pc8f6wk&st=1d68x9zt&dl=1" -O validate.zip
!unzip validate.zip -d data-rb-validate

In [7]:
import os
assert os.path.exists('data-rb-validate/02836284.webp')

In [ ]:
from datasets import load_from_disk, DatasetDict
dataset = load_from_disk('data-rb-large')
visual_validation_dataset = load_from_disk('data-rb-validate')

train_test_split = dataset.train_test_split(test_size=100/len(dataset)) 
train_dataset = train_test_split["train"]
temp_dataset = train_test_split["test"]

dataset = DatasetDict({
    "train": train_test_split["train"],
    "valid": visual_validation_dataset,
    "test": train_test_split["test"],
})

In [ ]:
dataset

In [8]:
model, tokenizer = load_model()

data_prompt = """Your job is to take an SVG file of a web design and convert it into a pixel-perfect HTML and CSS markup and stylesheet.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompt(examples):
    inputs       = examples["svg"]
    outputs      = examples["html"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }



==((====))==  Unsloth 2025.3.3: Fast Mllama vision patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA H100 NVL. Max memory: 93.111 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

Unsloth: Making `model.base_model.model.vision_model.transformer` require gradients


In [ ]:
training_data = dataset.map(formatting_prompt, batched=True)

In [ ]:
training_data

In [9]:
from datasets import load_from_disk

training_data = load_from_disk('data-rb-large-filtered-50000')

Loading dataset from disk:   0%|          | 0/190 [00:00<?, ?it/s]

In [10]:
def get_token_lengths(examples):
    inputs = tokenizer(
        examples['text'],
        truncation=False,  # Don't truncate yet
        padding=False,     # Don't pad yet
        return_length=True,
    )

    return inputs

tokenized_data = training_data.map(get_token_lengths, batched=True)

def filter_function(example):
    return example['length'] <= max_seq_length

filtered_data = tokenized_data.filter(filter_function)

print(filtered_data)

Map:   0%|          | 0/293102 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

Filter:   0%|          | 0/293102 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4 [00:00<?, ? examples/s]

Filter:   0%|          | 0/65 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['svg', 'html', 'text', 'input_ids', 'attention_mask', 'length'],
        num_rows: 100502
    })
    valid: Dataset({
        features: ['svg', 'html', 'text', 'input_ids', 'attention_mask', 'length'],
        num_rows: 4
    })
    test: Dataset({
        features: ['svg', 'html', 'text', 'input_ids', 'attention_mask', 'length'],
        num_rows: 23
    })
})


In [11]:
filtered_data = filtered_data.remove_columns(["input_ids", "attention_mask", "length"])
filtered_data.save_to_disk('data-rb-large-filtered-' + str(max_seq_length))

Saving the dataset (0/23 shards):   0%|          | 0/100502 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/23 [00:00<?, ? examples/s]

In [13]:
from datasets import load_from_disk

filtered_data = load_from_disk('data-rb-large-filtered-' + str(max_seq_length))


filtered_data

Loading dataset from disk:   0%|          | 0/23 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 100502
    })
    valid: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 4
    })
    test: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 23
    })
})

In [14]:
# Doing some reshuffling because:
# 1. The validation data set is arguably too simple for the 90B model and it does a pretty good job even before fine-tuning
# 2. The test data set is too small because of the filtering

from datasets import concatenate_datasets, DatasetDict

new_valid_data = concatenate_datasets([
    filtered_data['valid'],
    filtered_data['train'].select(range(25))
])

new_test_data = concatenate_datasets([
    filtered_data['test'],
    filtered_data['train'].select(range(25, 52))
])

new_train_data = filtered_data['train'].select(range(25, len(filtered_data['train'])))

filtered_data = DatasetDict({
    'train': new_train_data,
    'valid': new_valid_data,
    'test': new_test_data
})

In [15]:
filtered_data

DatasetDict({
    train: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 100477
    })
    valid: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 29
    })
    test: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 50
    })
})

In [ ]:
import torch
from tqdm import tqdm

resume = False

for steps in tqdm(range(0, 1500, 25)):
    print(f"Steps: {steps}")

    if steps > 0:
        trainer = create_trainer(model, tokenizer, filtered_data['train'], steps)
        if resume:
            trainer.train(resume_from_checkpoint=True)
        else:
            trainer.train()
            resume = True
        
    model = FastVisionModel.for_inference(model)

    results = test_prediction(model, filtered_data['valid'], steps)

    if results is not None and results['perceptual_loss'] == 0.0:
        break

    model = FastVisionModel.for_training(model)

    

  0%|          | 0/60 [00:00<?, ?it/s]

Steps: 0
Generating predictions...
Computing metrics...


  2%|▏         | 1/60 [1:03:42<62:39:00, 3822.72s/it]

Similarity: 0.9093418904598012
Perceptual loss: 0.5067523877489669
Steps: 25


Generating train split: 0 examples [00:00, ? examples/s]

...

Generating predictions...
Computing metrics...
Script failed with error: Traceback (most recent call last):
  File "/utils/similarity.py", line 192, in <module>
    take_screenshot(args.predicted_url, args.predicted_screenshot_path, args.viewport_width, args.viewport_height)
  File "/utils/similarity.py", line 96, in take_screenshot
    page = browser.new_page(viewport={'width': viewport_width, 'height': viewport_height})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/playwright/sync_api/_generated.py", line 14169, in new_page
    self._sync(
  File "/opt/conda/lib/python3.11/site-packages/playwright/_impl/_sync_base.py", line 115, in _sync
    return task.result()
           ^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/playwright/_impl/_browser.py", line 181, in new_page
    return await self._connection.wrap_api_call(inner)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

 20%|██        | 12/60 [228:31:10<979:27:07, 73458.91s/it] 

Script failed with error: Traceback (most recent call last):
  File "/utils/similarity.py", line 192, in <module>
    take_screenshot(args.predicted_url, args.predicted_screenshot_path, args.viewport_width, args.viewport_height)
  File "/utils/similarity.py", line 96, in take_screenshot
    page = browser.new_page(viewport={'width': viewport_width, 'height': viewport_height})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/playwright/sync_api/_generated.py", line 14169, in new_page
    self._sync(
  File "/opt/conda/lib/python3.11/site-packages/playwright/_impl/_sync_base.py", line 115, in _sync
    return task.result()
           ^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/playwright/_impl/_browser.py", line 181, in new_page
    return await self._connection.wrap_api_call(inner)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packag

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 45,811 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 128
\        /    Total batch size = 128 | Total steps = 300
 "-____-"     Number of trainable parameters = 265,420,800
🦥 Unsloth needs about 1-3 minutes to load everything - please wait!


Step,Training Loss
276,0.191200
277,0.232000
278,0.201200


In [11]:
test_index = 0
text = filtered_data['test'][test_index]['svg']
model = FastVisionModel.for_inference(model)
inputs = tokenizer(
[
    data_prompt.format(
        #instructions
        text,
        #answer
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = max_seq_length, use_cache = True)
answer=tokenizer.batch_decode(outputs)
answer = answer[0].split("### Response:")[-1]

print(filtered_data['test'][test_index]['svg'])
print(filtered_data['test'][test_index]['html'])
print("Answer of the question is:", answer)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="top bottom style1"><g data-tag="div" id="top" data-z-index="auto" data-stacking-context="true" aria-owns="top-left top-right"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="393" height="366.359375" x="0" y="0" fill="rgb(128, 5, 118)"/></g><g data-tag="div" id="top-left" data-z-index="auto" data-stacking-context="true"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="200.421875" height="366.359375" x="0" y="0" fill="rgb(113, 7, 188)"/></g><text color="rgb(184, 124, 151)" dominant-baseline="text-after-edge" font-family="&quot;Tim

In [ ]:
test_prediction(model, filtered_data['test'], steps+1)

In [18]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize the dataset
tokenized_dataset = filtered_data['train'].select(range(1000)).map(tokenize_function, batched=True)

df = tokenized_dataset.to_pandas()
nan_counts = df.isna().sum()
print(f"NaN counts in split:\n", nan_counts)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

NaN counts in split:
 svg               0
html              0
text              0
input_ids         0
attention_mask    0
dtype: int64
